Установка - Image/Video Super-Resolution

Uses [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) to perform neural super-resolution (creates HD images or videos from low-resolution).

The modelds were fine-tuned by russian company [Sber](https://github.com/sberbank-ai/ru-dalle).

![Example](https://pollinations.ai/ipfs/QmaSXnegTaivkmcFKjDT2oRewiswsAac3gJk8mEWDf3Cbn)

In [5]:

# The image or video to scale up
video_file = ''  #@param {type: "string"}

# By what amount to upscale the image
upscale_factor = "x4"  #@param ['x2', 'x4', 'x8']

output_path = "/content"

In [ ]:
import mimetypes
mimetypes.init()

mimestart = mimetypes.guess_type("test.mp3")[0]
mimestart = mimestart.split('/')[0]
is_video = mimestart == 'video'

input_path = "/tmp/sr_input"
!mkdir -p $input_path

# Copy image or extract video frames and copy to input_path

if is_video:
    # Save video framerate
    framerate = !ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of default=noprint_wrappers=1:nokey=1 $video_file
    print("framerate", framerate)

    # Extract frames
    !ffmpeg -i $video_file $input_path/%05d.png
else:
    !cp $video_file $input_path/





Веса для x2, x4, x8 моделей можно скачать по этой ссылке:
https://drive.google.com/drive/folders/16PlVKhTNkSyWFx52RPb2hXPIQveNGbxS

In [2]:

model_file = f"SBER_RealESRGAN_{upscale_factor}.pth"
model_url = f"https://pollinations.ai/ipfs/QmbgtAqZyBTUhyUFh6ZZD5HbgWJBSNnnPrzj72k4ikq7PV/{model_file}"

!git clone https://github.com/sberbank-ai/Real-ESRGAN
%cd /content/Real-ESRGAN
!pip install -r requirements.txt

!wget -N {model_url}

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 20 (delta 2), reused 17 (delta 2), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/Real-ESRGAN
--2021-11-04 05:13:26--  https://pollinations.ai/ipfs/QmbgtAqZyBTUhyUFh6ZZD5HbgWJBSNnnPrzj72k4ikq7PV/SBER_RealESRGAN_x4.pth
Resolving pollinations.ai (pollinations.ai)... 50.18.120.10, 138.197.207.178, 2604:a880:2:d0::212a:f001, ...
Connecting to pollinations.ai (pollinations.ai)|50.18.120.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67040989 (64M) [application/zip]
Saving to: ‘SBER_RealESRGAN_x4.pth’

SBER_RealESRGAN_x4. 100%[===================>]  63.93M  20.8MB/s    in 3.5s    

2021-11-04 05:13:30 (18.2 MB/s) - ‘SBER_RealESRGAN_x4.pth’ saved [67040989/67040989]



# Загрузка изображений/архивов


# Запуск модели

In [6]:
# Создание модели
from realesrgan import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

scale_map = {
    "x2":2, "x4":4, "x8":8
}

model = RealESRGAN(device, scale=scale_map[upscale_factor])
model.load_weights(model_file)

device: cuda


Улучшение одного изображения:

In [7]:

image = Image.open(video_file).convert('RGB')
sr_image = model.predict(np.array(image))

sr_image.save(f"{output_path}/upscaled.png")